# Steady State - Aggregate

**Table of contents**<a id='toc0_'></a>    
- 1. [Setup](#toc1_)    
- 2. [Find steady state](#toc2_)    
  - 2.1. [Compare with data](#toc2_1_)    
- 3. [Speed and error tests](#toc3_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
%load_ext autoreload
%autoreload 2

import time
import pickle
import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt   
plt.rcParams.update({"axes.grid":True,"grid.color":"black","grid.alpha":"0.25","grid.linestyle":"--"})
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

from BabyMAKROModel import BabyMAKROModelClass

## 1. <a id='toc1_'></a>[Setup](#toc0_)

In [2]:
model = BabyMAKROModelClass(name='baseline') # create an instance of the model

## 2. <a id='toc2_'></a>[Find steady state](#toc0_)

In [3]:
par = model.par
ss = model.ss

model.find_ss(do_print=True)    

Labor supply, search and matching:
ss.S/par.N_work = 0.16, ss.L/par.N_work = 0.96, ss.U/par.N_work = 0.04
ss.delta_L = 0.12, ss.m_s = 0.75, ss.m_v = 0.75, ss.v = 6.67
par.sigma_m = 0.42
Capital agency FOC:
ss.r_K = 0.12
Labor agency FOC:
ss.r_ell = 1.01, (ss.LH-ss.ell)/par.N_work*100 = 0.78


Production firm:
ss.P_Y_0 = 0.91, ss.Gamma = 0.54, ss.Y = 125.59, ss.K = 321.66
Capital accumulation:
ss.iota = 32.17, ss.I = 32.17
Government:
ss.B = 0.00, ss.G = 31.40, ss.tau = 0.48
Households:
ss.Aq/par.N = 0.06, ss.real_W = 1.00, ss.C = 44.72, ss.A = 29.59, ss.r_hh = 0.02
Market clearing:
ss.C/ss.Y = 0.36, ss.G/ss.Y = 0.25, ss.I/ss.Y = 0.26, ss.X/ss.Y = 0.60, ss.M/ss.Y = 0.46


### 2.1. <a id='toc2_1_'></a>[Compare with data](#toc0_)

In [4]:
with open('data/ss_aggregate.pickle', 'rb') as f:
    ss_aggregate = pickle.load(f)

for k, v in ss_aggregate.items():
    print(f'{k}')
    print(f'  From data: {v:.2f}')
    print(f'  From model: {getattr(ss, k):.2f}')

M_ratio
  From data: 0.46
  From model: 0.46
X_ratio
  From data: 0.52
  From model: 0.60
C_ratio
  From data: 0.47
  From model: 0.36
G_ratio
  From data: 0.25
  From model: 0.25
I_ratio
  From data: 0.22
  From model: 0.26
K_ratio
  From data: 5.44
  From model: 2.56
L_ratio
  From data: 0.74
  From model: 0.72
U
  From data: 3.93
  From model: 1.67


## 3. <a id='toc3_'></a>[Speed and error tests](#toc0_)

In [5]:
model.set_exo_ss()          
model.set_unknowns_ss()     

**Python evaluation:**

In [6]:
%time model.evaluate_blocks(py=True)

CPU times: total: 6.91 s
Wall time: 7.12 s


**Check errors:**

In [7]:
errors = model.get_errors(do_print=True)

A_R_ini_error       : abs. max = 8.93e-15
Aq_diff             : abs. max = 2.47e-14
FOC_capital_agency  : abs. max = 2.22e-16
FOC_K_ell           : abs. max = 5.33e-15
mkt_clearing        : abs. max = 2.84e-14
PC                  : abs. max = 0.00e+00


In [8]:
if np.allclose(errors,0.0):
    print('errors are zero')
else:
    raise ValueError('errors are NOT zero')

errors are zero


**First numba evaluation:**

In [9]:
%time model.evaluate_blocks()

CPU times: total: 1min 50s
Wall time: 1min 51s


In [10]:
errors = model.get_errors(do_print=True)

A_R_ini_error       : abs. max = 8.93e-15
Aq_diff             : abs. max = 2.47e-14
FOC_capital_agency  : abs. max = 2.22e-16
FOC_K_ell           : abs. max = 5.33e-15
mkt_clearing        : abs. max = 2.84e-14
PC                  : abs. max = 0.00e+00


**Time:**

In [11]:
%timeit model.evaluate_blocks()
%timeit model.get_errors()

40.2 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


36.5 µs ± 1.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [12]:
%timeit model.evaluate_blocks(py=True)

2.44 s ± 28.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
